In [88]:
import pandas as pd
import numpy as np
from sklearn import linear_model, preprocessing
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from scipy.spatial.distance import cdist
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.exceptions import ConvergenceWarning
import xgboost
from sklearn.decomposition import PCA
import warnings
import random
import time
import os
import sys
import argparse
import torch
from sklearn.cluster import KMeans
from sklearn_extra.cluster import KMedoids
from Bio import SeqIO

# Ignore FutureWarnings and ConvergenceWarnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning, module="sklearn.neural_network")
pd.options.mode.chained_assignment = None  # default='warn'


### Functions ported from grid_search

In [89]:
# Function to read in the data
def read_data(dataset_name, base_path, file_type, embeddings_type='both', experimental = False):
    # Construct the file paths
    if file_type == "csvs":
        labels_file = os.path.join(base_path, 'labels', dataset_name.split('_')[0] + '_labels.csv')
        hie_file = os.path.join(base_path, 'hie_temp', dataset_name.split('_')[0] + '.csv')
        embeddings_file = os.path.join(base_path, 'csvs', dataset_name + '.csv')
        # Read in mean embeddings across all rounds
        embeddings = pd.read_csv(embeddings_file, index_col=0)
    elif file_type == "pts":
        labels_file = os.path.join(base_path, 'labels', dataset_name.split('_')[-1] + '_labels.csv')
        hie_file = os.path.join(base_path, 'hie_temp', dataset_name.split('_')[-1] + '.csv')
        embeddings_file = os.path.join(base_path, 'pts', dataset_name + '.pt')
        # Read in pytorch tensor of embeddings
        embeddings = torch.load(embeddings_file)
        # Convert embeddings to a dataframe
        if embeddings_type == 'average':
            embeddings = {key: value['average'].numpy() for key, value in embeddings.items()}
        elif embeddings_type == 'mutated':
            embeddings = {key: value['mutated'].numpy() for key, value in embeddings.items()}
        elif embeddings_type == 'both':
            embeddings = {key: torch.cat((value['average'], value['mutated'])).numpy() for key, value in embeddings.items()}
        else:
            print("Invalid embeddings_type. Please choose 'average', 'mutated', or 'both'")
            return None, None

        # Convert embeddings dictionary to a dataframe
        embeddings = pd.DataFrame.from_dict(embeddings, orient='index')
    else:
        print("Invalid file type. Please choose either 'csvs' or 'pts'")
        return None, None

    # if not experimental
    if not experimental:
        # Read in labels
        labels = pd.read_csv(labels_file)

        # Read in hierarchy
        hie_data = pd.read_csv(hie_file)

        # Filter out rows where fitness is NaN
        labels = labels[labels['fitness'].notna()]

        # Filter out rows in embeddings where row names are not in labels variant column
        embeddings = embeddings[embeddings.index.isin(labels['variant'])]

        # Align labels by variant
        labels = labels.sort_values(by=['variant'])

        # Align embeddings by row name
        embeddings = embeddings.sort_index()

        # Confirm that labels and embeddings are aligned, reset index
        labels = labels.reset_index(drop=True)

        # Get the variants in labels and embeddings, convert to list
        label_variants = labels['variant'].tolist()
        embedding_variants = embeddings.index.tolist()

        # Check if embedding row names and label variants are identical
        if label_variants == embedding_variants:
            print('Embeddings and labels are aligned')

        # return embeddings and labels
        return embeddings, labels, hie_data

    else:
        return embeddings


# Active learning function for one iteration
def top_layer(iter_train, iter_test, embeddings_pd, labels_pd, measured_var, regression_type='ridge', top_n=None, final_round=10):
    # reset the indices of embeddings_pd and labels_pd
    embeddings_pd = embeddings_pd.reset_index(drop=True)
    labels_pd = labels_pd.reset_index(drop=True)

    # save column 'iteration' in the labels dataframe
    iteration = labels_pd['iteration']

    # save labels
    labels = labels_pd

    # save mean embeddings as numpy array
    a = embeddings_pd

    # subset a, y to only include the rows where iteration = iter_train and iter_test
    idx_train = iteration[iteration.isin(iter_train)].index.to_numpy()
    idx_test = iteration[iteration.isin([iter_test])].index.to_numpy()

    # subset a to only include the rows where iteration = iter_train and iter_test
    X_train = a.loc[idx_train, :]
    X_test = a.loc[idx_test, :]

    y_train = labels[iteration.isin(iter_train)][measured_var]

    y_test = labels[iteration.isin([iter_test])][measured_var]

    # fit
    if regression_type == 'ridge':
        model = linear_model.RidgeCV()
    elif regression_type == 'lasso':
        model = linear_model.LassoCV(max_iter=100000,tol=1e-3)
    elif regression_type == 'elasticnet':
        model = linear_model.ElasticNetCV(max_iter=100000,tol=1e-3)
    elif regression_type == 'linear':
        model = linear_model.LinearRegression()
    elif regression_type == 'neuralnet':
        model = MLPRegressor(hidden_layer_sizes=(5), max_iter=1000, activation='relu', solver='adam', alpha=0.001,
                             batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5,
                             momentum=0.9, nesterovs_momentum=True, shuffle=True, random_state=1, tol=0.0001,
                             verbose=False, warm_start=False, early_stopping=False, validation_fraction=0.1, beta_1=0.9,
                             beta_2=0.999, epsilon=1e-08)
    elif regression_type == 'randomforest':
        model = RandomForestRegressor(n_estimators=100, criterion='friedman_mse', max_depth=None, min_samples_split=2,
                                      min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto',
                                      max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False,
                                      n_jobs=None, random_state=1, verbose=0, warm_start=False, ccp_alpha=0.0,
                                      max_samples=None)
    elif regression_type == 'gradientboosting':
        model = xgboost.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.3, learning_rate=0.1,
                                     max_depth=5, alpha=10, n_estimators=10)

    model.fit(X_train, y_train)

    # make predictions on train data
    y_pred_train = model.predict(X_train)
    y_std_train = np.zeros(len(y_pred_train))
    # make predictions on test data
    # NOTE: can work on alternate 2-n round strategies here
    y_pred_test = model.predict(X_test)
    y_std_test = np.zeros(len(y_pred_test))

    # combine predicted and actual thermostability values with sequence IDs into a new dataframe
    df_train = pd.DataFrame({'variant': labels.variant[idx_train], 'y_pred': y_pred_train, 'y_actual': y_train})
    df_test = pd.DataFrame({'variant': labels.variant[idx_test], 'y_pred': y_pred_test, 'y_actual': y_test})
    
    # sort df_test by y_pred
    df_test = df_test.sort_values(by=['y_pred'], ascending=False)

    df_all = pd.concat([df_train, df_test])

    # sort df_all by y_pred
    df_all = df_all.sort_values(by=['y_pred'], ascending=False)

    return df_test, df_all

### New functions

In [90]:
def read_experimental_data(base_path, round_file_name, t7_sequence):
    file_path = base_path + '/rounds/' + round_file_name
    df = pd.read_excel(file_path)

    # Clean up 'Variant' column
    df['Variant'] = df['Variant'].str.replace('T7pol_', '')

    # Iterate through the 'Variant' column and update the values based on t7_sequence
    updated_variants = []
    for _, row in df.iterrows():
        variant = row['Variant']
        if variant == 'WT':
            updated_variants.append(variant)
        else:
            position = int(variant[:-1])
            wt_aa = t7_sequence[position - 1]
            updated_variant = wt_aa + variant
            updated_variants.append(updated_variant)
    
    df['updated_variant'] = updated_variants  # Add the updated variants to the DataFrame
    
    return df

def create_dataframes(df, expected_index):
    # First dataframe
    df1 = df.copy()
    df1.loc[df1['updated_variant'] == 'WT', 'iteration'] = 0
    df1.loc[df1['updated_variant'] != 'WT', 'iteration'] = 1
    df1['iteration'] = df1['iteration'].astype(int)
    df1.rename(columns={'updated_variant': 'variant'}, inplace=True)  # Rename the column
    df1 = df1[['variant', 'iteration']]
    
    # Second dataframe
    df2 = df.copy()
    df2.rename(columns={'Effect vector': 'fitness'}, inplace=True)  # Rename the column
    df2.loc[df2['updated_variant'] == 'WT', 'iteration'] = 0
    df2.loc[df2['updated_variant'] != 'WT', 'iteration'] = 1
    df2.rename(columns={'updated_variant': 'variant'}, inplace=True)  # Rename the column
    df2['iteration'] = df2['iteration'].astype(int)
    df2 = df2[['variant', 'fitness', 'iteration']]

    expected_index = [variant for variant in expected_index if variant not in df2['variant'].tolist()]
    # make a df_external that has a column 'variant' with all the variants in expected_index
    df_external = pd.DataFrame({'variant': expected_index})
    df_external = pd.DataFrame({'variant': expected_index})
    df_external['fitness'] = np.nan  
    df_external['iteration'] = 1001 
    df2 = df2.append(df_external, ignore_index=True)

    return df1, df2

In [91]:
# import brenan data
dataset_name = 'esm2_15B_t7_pol'
base_path = '/Users/matteodibernardo/Documents/GitHub/directed_evolution/notebooks/t7/'
file_type = 'pts'
embeddings_type = 'average'
experimental = True
embeddings = read_data(dataset_name, base_path, file_type, embeddings_type, experimental)
# replace WT Wild-type sequence index in embeddings with 'WT'
embeddings.index.values[0] = 'WT'

In [92]:
embeddings

,0,1,2,3,4,5,6,7,8,9,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
WT,0.147168,-0.183875,0.031651,-0.052828,-0.141786,-0.013741,-0.061830,-0.028053,0.018626,0.025174,...,-0.110809,0.051659,0.059534,-0.157181,0.013038,0.020313,0.015764,0.064493,-0.008039,-0.108970
M1A,0.147994,-0.184054,0.035075,-0.053447,-0.140782,-0.016417,-0.063300,-0.028770,0.018573,0.022787,...,-0.111060,0.049681,0.060578,-0.156948,0.011523,0.018843,0.014856,0.063135,-0.005746,-0.106902
M1C,0.149553,-0.182680,0.033869,-0.054389,-0.140723,-0.015113,-0.064822,-0.028185,0.018523,0.027269,...,-0.111689,0.051024,0.059965,-0.157671,0.012664,0.019815,0.015817,0.063816,-0.007434,-0.108856
M1D,0.146734,-0.182717,0.034278,-0.052991,-0.140832,-0.015275,-0.063740,-0.028154,0.018495,0.025465,...,-0.111113,0.050927,0.060537,-0.156441,0.011660,0.019746,0.013886,0.063376,-0.006656,-0.107104
M1E,0.149375,-0.184232,0.034520,-0.054731,-0.140585,-0.016518,-0.064851,-0.028408,0.018998,0.025971,...,-0.110941,0.051635,0.058811,-0.157911,0.012537,0.020981,0.014563,0.062794,-0.007955,-0.106604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A883S,0.145864,-0.184623,0.031468,-0.051257,-0.142275,-0.012640,-0.061425,-0.027589,0.017693,0.025046,...,-0.110233,0.051852,0.058792,-0.157249,0.013417,0.018916,0.019343,0.065511,-0.009308,-0.109120
A883T,0.145933,-0.184290,0.031539,-0.051148,-0.141504,-0.012715,-0.062040,-0.028451,0.017504,0.024164,...,-0.110211,0.052196,0.058730,-0.158339,0.012717,0.018639,0.020539,0.065699,-0.010361,-0.109183
A883V,0.145253,-0.182491,0.031516,-0.052928,-0.142025,-0.013264,-0.060926,-0.027928,0.017077,0.024314,...,-0.111040,0.051419,0.060660,-0.157816,0.012737,0.018943,0.017333,0.065630,-0.008499,-0.109677
A883W,0.145614,-0.183985,0.031491,-0.051818,-0.142407,-0.012759,-0.059910,-0.027657,0.016999,0.023699,...,-0.110127,0.052603,0.059757,-0.156924,0.013284,0.018394,0.016497,0.064918,-0.009098,-0.108907


In [93]:
base_path = '/Users/matteodibernardo/Documents/GitHub/directed_evolution/notebooks/t7/'
round_file_name = 'T7_Round1.xlsx'
t7_sequence = 'MNTINIAKNDFSDIELAAIPFNTLADHYGERLAREQLALEHESYEMGEARFRKMFERQLKAGEVADNAAAKPLITTLLPKMIARINDWFEEVKAKRGKRPTAFQFLQEIKPEAVAYITIKTTLACLTSADNTTVQAVASAIGRAIEDEARFGRIRDLEAKHFKKNVEEQLNKRVGHVYKKAFMQVVEADMLSKGLLGGEAWSSWHKEDSIHVGVRCIEMLIESTGMVSLHRQNAGVVGQDSETIELAPEYAEAIATRAGALAGISPMFQPCVVPPKPWTGITGGGYWANGRRPLALVRTHSKKALMRYEDVYMPEVYKAINIAQNTAWKINKKVLAVANVITKWKHCPVEDIPAIEREELPMKPEDIDMNPEALTAWKRAAAAVYRKDKARKSRRISLEFMLEQANKFANHKAIWFPYNMDWRGRVYAVSMFNPQGNDMTKGLLTLAKGKPIGKEGYYWLKIHGANCAGVDKVPFPERIKFIEENHENIMACAKSPLENTWWAEQDSPFCFLAFCFEYAGVQHHGLSYNCSLPLAFDGSCSGIQHFSAMLRDEVGGRAVNLLPSETVQDIYGIVAKKVNEILQADAINGTDNEVVTVTDENTGEISEKVKLGTKALAGQWLAYGVTRSVTKRSVMTLAYGSKEFGFRQQVLEDTIQPAIDSGKGLMFTQPNQAAGYMAKLIWESVSVTVVAAVEAMNWLKSAAKLLAAEVKDKKTGEILRKRCAVHWVTPDGFPVWQEYKKPIQTRLNLMFLGQFRLQPTINTNKDSEIDAHKQESGIAPNFVHSQDGSHLRKTVVWAHEKYGIESFALIHDSFGTIPADAANLFKAVRETMVDTYESCDVLADFYDQFADQLHESQLDKMPALPAKGNLNLRDILESDFAFA'
experimental_data = read_experimental_data(base_path, round_file_name, t7_sequence)
print(experimental_data)

  Variant  Effect vector updated_variant
0     12N       1.073846            S12N
1     25N       0.677227            A25N
2      WT       1.000000              WT
3     89R       0.740499            F89R
4    134T       1.074891           V134T
5    177L       1.042706           V177L
6    225E       1.075861           G225E
7    241W       0.938351           S241W
8    273H       0.785147           V273H


In [94]:
iterations_one, labels_one = create_dataframes(experimental_data, embeddings.index)

In [95]:
iteration_old = iterations_one
embeddings_pd = embeddings
labels_pd = labels_one
measured_var = 'fitness'
regression_type = 'randomforest'
num_mutants_per_round = 16
final_round = 16

df_test, df_all = top_layer(
    iter_train=iteration_old['iteration'].unique().tolist(),
    iter_test=1001,
    embeddings_pd=embeddings_pd,
    labels_pd=labels_pd,
    measured_var=measured_var,
    regression_type=regression_type,
    top_n=None,
    final_round=final_round
)

In [96]:
df_test

,variant,y_pred,y_actual
4715,E249C,1.011400,NaN
5569,L294C,1.010837,NaN
8905,G469Q,1.010637,NaN
5298,T279S,1.009233,NaN
5329,I281L,1.008760,NaN
...,...,...,...
16,M1I,0.907080,NaN
15452,F814G,0.906483,NaN
10,M1C,0.906116,NaN
12,M1E,0.905141,NaN


In [99]:
# write the dataframe to a csv file
df_test.to_csv('t7/round1_predictions.csv', index=False)

In [98]:
df_all

,variant,y_pred,y_actual
0,S12N,1.022667,1.073846
4,V134T,1.011421,1.074891
4715,E249C,1.011400,NaN
5569,L294C,1.010837,NaN
8905,G469Q,1.010637,NaN
...,...,...,...
12,M1E,0.905141,NaN
10360,F546G,0.901978,NaN
8,V273H,0.835153,0.785147
3,F89R,0.799308,0.740499
